<a href="https://colab.research.google.com/github/mariliabrito/gunviolence/blob/main/Viol%C3%AAncia_Escolar.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

*texto em itálico*<a href="TAIL"><img src="https://i.imgur.com/GRTE0OG.png" width="150" title="Tail Logo" /></a>

# <font color ="#00b8ff">Análise de Violência Escolar</font>
## <font color ="#00b8ff">Trainees - 2022.2</font>

## <font color ="#00b8ff">1. Introdução</font>

Fornecer um contexto do problema, trazer dados sobre violência escolar com armas, esclarecer os objetivos do trabalho, informar de onde pegamos o notebook e sua credibilidade, descrever as colunas do dataset, etc

In [ ]:
import pandas as pd

In [ ]:
# Importando 4 abas do arquivo

df_incid = pd.read_excel("https://github.com/mariliabrito/gunviolence/blob/main/Public%20v3.1%20K-12%20School%20Shooting%20Database.xlsx?raw=true", sheet_name=1)
df_shoot = pd.read_excel("https://github.com/mariliabrito/gunviolence/blob/main/Public%20v3.1%20K-12%20School%20Shooting%20Database.xlsx?raw=true", sheet_name=2)
df_victm = pd.read_excel("https://github.com/mariliabrito/gunviolence/blob/main/Public%20v3.1%20K-12%20School%20Shooting%20Database.xlsx?raw=true", sheet_name=3)
df_weapn = pd.read_excel("https://github.com/mariliabrito/gunviolence/blob/main/Public%20v3.1%20K-12%20School%20Shooting%20Database.xlsx?raw=true", sheet_name=4)

## <font color ="#00b8ff">2. Limpeza</font>

Realizar limpeza e uniformização do dataset. Também é possível juntar datasets pela coluna incidentid.

## <font color ="#00b8ff">3. Análise Descritiva</font>

Realizar análise do dataset, mostrar série histórica, trazer visualizações, etc

### <font color ="#00b8ff">3.1 Série histórica</font>

### <font color ="#00b8ff">3.2 Análise dos Incidentes</font>

### <font color ="#00b8ff">3.3 Análise do Atirador</font>

### <font color ="#00b8ff">3.4 Análise das Vítimas</font>

### <font color ="#00b8ff">3.5 Análise das Armas</font>